In [4]:
!pip install pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Read CSV").getOrCreate()

# Read csv. Escape must be set (see next cell)
df = spark.read.csv("/content/merged_data_10000.csv", header=True, inferSchema=True, escape="\"")

In [5]:
# Problem row. If escape is not set, the commas in 'artist' count as new columns and push data into columns to the right
# In this case, the 'url' was pushed into 'trend'
df.collect()[1359]

Row(_c0=1359, title='Bailen (Remix)', rank=87, date='1/1/2017', artist='Franco "El Gorilla", De La Ghetto, Luigi 21 Plus, Ozuna', url='https://open.spotify.com/track/3dqBmUFihufaWii7TGLXyE', region='Chile', chart='top200', trend='MOVE_UP', streams=19176, track_id='3dqBmUFihufaWii7TGLXyE', album='Reloaded', popularity=0, duration_ms=223861, explicit=True, release_date='7/15/2016', available_markets='[]', af_danceability=0.733, af_energy=0.857, af_key=7, af_loudness=-3.454, af_mode=0, af_speechiness=0.0846, af_acousticness=0.134, af_instrumentalness=0.0, af_liveness=0.208, af_valence=0.677, af_tempo=176.004, af_time_signature=4)

In [6]:
# Print the schema to understand the data types and column names
df.show(5)

+---+--------------------+----+--------+--------------------+--------------------+---------+------+-------------+-------+--------------------+--------------------+----------+-----------+--------+------------+--------------------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+
|_c0|               title|rank|    date|              artist|                 url|   region| chart|        trend|streams|            track_id|               album|popularity|duration_ms|explicit|release_date|   available_markets|af_danceability|af_energy|af_key|af_loudness|af_mode|af_speechiness|af_acousticness|af_instrumentalness|af_liveness|af_valence|af_tempo|af_time_signature|
+---+--------------------+----+--------+--------------------+--------------------+---------+------+-------------+-------+--------------------+--------------------+----------+-----------+--------+------------+--------------------+---

In [9]:
# Print the schema to understand the data types and column names
df.schema

StructType([StructField('_c0', IntegerType(), True), StructField('title', StringType(), True), StructField('rank', IntegerType(), True), StructField('date', StringType(), True), StructField('artist', StringType(), True), StructField('url', StringType(), True), StructField('region', StringType(), True), StructField('chart', StringType(), True), StructField('trend', StringType(), True), StructField('streams', IntegerType(), True), StructField('track_id', StringType(), True), StructField('album', StringType(), True), StructField('popularity', IntegerType(), True), StructField('duration_ms', IntegerType(), True), StructField('explicit', BooleanType(), True), StructField('release_date', StringType(), True), StructField('available_markets', StringType(), True), StructField('af_danceability', DoubleType(), True), StructField('af_energy', DoubleType(), True), StructField('af_key', IntegerType(), True), StructField('af_loudness', DoubleType(), True), StructField('af_mode', IntegerType(), True),

In [10]:
# Summary statistics for numerical columns
df.describe().show()

+-------+------------------+--------------------+------------------+--------+------------+--------------------+---------+------+-------------+------------------+--------------------+--------------------+-----------------+------------------+------------------+--------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|summary|               _c0|               title|              rank|    date|      artist|                 url|   region| chart|        trend|           streams|            track_id|               album|       popularity|       duration_ms|      release_date|   available_markets|   af_danceability|          af_energy|            af_key|       af_loudness|           af_mode|     af_speechiness|    af_acousticness|af_instrumentalness|        af_liveness|         af_valence|        

In [44]:
# Check 'af_' values for any text. If any text appears, other columns have shifted over
from pyspark.sql import functions as F

column_af = [ 'af_danceability', 'af_energy', 'af_key', 'af_loudness', 'af_mode', 'af_speechiness', 'af_acousticness',
             'af_instrumentalness', 'af_liveness', 'af_valence', 'af_tempo', 'af_time_signature']

# Displays non-numbers if they show up in the column. For proof, see cell below
for column in column_af:
  temp_df = df.select(column, F.col(column).cast("int").isNotNull().alias("IsInt"))
  temp_df.filter(temp_df.IsInt==False).show()


+---------------+-----+
|af_danceability|IsInt|
+---------------+-----+
+---------------+-----+

+---------+-----+
|af_energy|IsInt|
+---------+-----+
+---------+-----+

+------+-----+
|af_key|IsInt|
+------+-----+
+------+-----+

+-----------+-----+
|af_loudness|IsInt|
+-----------+-----+
+-----------+-----+

+-------+-----+
|af_mode|IsInt|
+-------+-----+
+-------+-----+

+--------------+-----+
|af_speechiness|IsInt|
+--------------+-----+
+--------------+-----+

+---------------+-----+
|af_acousticness|IsInt|
+---------------+-----+
+---------------+-----+

+-------------------+-----+
|af_instrumentalness|IsInt|
+-------------------+-----+
+-------------------+-----+

+-----------+-----+
|af_liveness|IsInt|
+-----------+-----+
+-----------+-----+

+----------+-----+
|af_valence|IsInt|
+----------+-----+
+----------+-----+

+--------+-----+
|af_tempo|IsInt|
+--------+-----+
+--------+-----+

+-----------------+-----+
|af_time_signature|IsInt|
+-----------------+-----+
+--------------

In [45]:
temp_df = df.select('title', F.col('title').cast("int").isNotNull().alias("IsInt"))
temp_df.filter(temp_df.IsInt==False).show()

+-----+-----+
|title|IsInt|
+-----+-----+
|  743| true|
|  743| true|
| 1955| true|
|   47| true|
|  777| true|
| 1955| true|
+-----+-----+

